In [4]:
from langgraph.graph import StateGraph, START, END, add_messages
from typing import TypedDict, Annotated, Literal, List, Optional
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
from jsonschema import validate
from pydantic import BaseModel, Field
import json
from prompts.prompts_config import PromptConfig

In [5]:
config = PromptConfig()

In [6]:

config.get_prompt(
    "fix_code_after_code_review",
    feedback="No Change",
    original_code="check.py"
    
)

'You are an experienced Software Engineer tasked with updating code after a code review.  \nYou will receive the **original source code** along with **feedback from a code review**.  \nYour task is to apply the feedback and return the revised code.  \n\n---\n\n#### Input\n- **Original Code:** The implementation before the review.  \n- **Review Feedback:** A list of issues, improvements, or changes suggested during code review.  \n\n---\n\n#### Instructions\n1. Carefully read the feedback and identify all required changes.  \n2. Apply each change to the original code while preserving its intended functionality.  \n3. Ensure that the updated code:  \n  - Fixes the issues mentioned in the feedback.  \n  - Remains clean, modular, and maintainable.  \n  - Does not introduce new bugs or unnecessary complexity.  \n4. Do **not** generate explanations, reasoning, or test cases.  \n5. Return only the revised source code.  \n\n---\n\n#### Output Format\n- Provide the **updated code only** inside 

In [7]:
load_dotenv()

True

In [8]:
model = ChatGroq(model="llama-3.1-8b-instant")
# model = ChatOpenAI(model="gpt-4o-mini")

### Utility Functions

In [9]:
def get_evaluator_schema(model):

    class EvaluatorSchema(BaseModel):
        status: Literal["approved", "feedback"]
        feedback: Optional[List[str]] = None
        
    structured_model = model.with_structured_output(EvaluatorSchema)
    
    return structured_model


In [10]:
class SDLCState(TypedDict):
    user_input_requirements: str
    auto_generated_user_stories_markdown: Annotated[list[str], add_messages]
    design_docs: Annotated[list[str], add_messages]
    code: str
    code_review_response: str

### Node Functions

In [11]:
def user_input_requirements(state: SDLCState):
    user_input = """
        I want to build a task management web application. 
        Features:
        - Users can create, update, and delete tasks.
        - Tasks should have priority and due dates.
        - A dashboard to track completed vs pending tasks.
        - Simple login with email and password.
        Tech Stack Preference: Python (FastAPI) for backend, React for frontend.
        Deployment: Host on AWS.
        """
    
    return {"user_input_requirements": user_input}

In [12]:
def auto_generated_user_stories(state: SDLCState):
    prompt_template = PromptTemplate(
        template="""
         You are an expert Agile Product Owner and Business Analyst. 
         Your task is to generate high-quality, clear, and structured *user stories* from the given software requirements.  

         ### Input
         You will receive:
         - A description of user requirements (provided by the user).
         - The project context, if available.

         ### Instructions
         1. Break down the given requirements into **user stories** following the Agile format:  
            *As a [role], I want [feature] so that [benefit]*.
            
         2. Ensure that each user story:
            - Captures a single functionality or feature.  
            - Clearly identifies the **user role** (end-user, admin, developer, etc.).  
            - Describes the **action or feature** in simple language.  
            - States the **value or outcome** of the feature.  

         3. For each user story, provide:  
            - **User Story ID** (US-1, US-2, …)  
            - **Story Statement** (in Agile format)  
            - **Acceptance Criteria** (written as Given/When/Then or bullet points)  
            - **Priority** (High, Medium, Low)  
            - **Dependencies** (if any)  

         4. If requirements are ambiguous or incomplete:
            - Highlight unclear areas in a separate section called **Clarification Questions**.  

         ### Output Format
         Return the output in **well-structured Markdown** with the following sections:  
         - **User Stories** (list each with ID, statement, acceptance criteria, priority, dependencies)  
         - **Clarification Questions** (if needed)  

         ### Example
         **User Story (US-1):**  
         As a registered user, I want to reset my password so that I can regain access to my account when I forget it.  

         **Acceptance Criteria:**  
         - Given that the user is on the login page, when they click "Forgot Password", then they should be prompted to enter their email.  
         - An email with a reset link should be sent.  
         - The reset link should expire after 24 hours.  

         **Priority:** High  
         **Dependencies:** Email service setup  

         ---
         
         User Input: \n {user_input}
         
         ---

         Now, based on the provided requirements, generate a complete set of user stories.
   """,
        input_variables=["user_input"],
    )

    user_story_chain = prompt_template | model | StrOutputParser()

    markdown_response = user_story_chain.invoke(
        {"user_input": state["user_input_requirements"]}
    )

    return {"auto_generated_user_stories_markdown": markdown_response}

In [13]:
def product_owner_review():
    pass

In [14]:
def create_design_docs(state: SDLCState):
    prompt_template = PromptTemplate(
        template="""
        You are a Senior Software Architect.  
         Your task is to generate structured **design documentation** from the approved user stories.  
         The documentation must cover both **Functional Requirements (FRs)** and **Technical Requirements (TRs)** in detail.  

         ### Input
         You will receive:
         - A set of approved user stories (with ID, statement, acceptance criteria, priority, dependencies).  

         ### Instructions
         1. **Functional Requirements (FRs):**
            - For each user story, identify the main functionality.  
            - Document the system behavior in terms of inputs, processes, and outputs.  
            - Link each FR to the corresponding User Story ID.  
            - Write requirements in clear, testable language (avoid vague terms).  

            *Example:*  
            FR-1 (Linked to US-2): The system shall allow users to reset their password via a secure email link.  

         2. **Technical Requirements (TRs):**
            - Define the technical aspects needed to implement each FR.  
            - Include details such as:  
            - System architecture choices (e.g., client-server, microservices).  
            - API design or endpoints.  
            - Database design considerations (tables, entities, relationships).  
            - Security requirements (encryption, authentication, authorization).  
            - Performance requirements (response time, scalability).  
            - Integration points (third-party services, APIs).  
            - Link each TR to the related FR(s).  

            *Example:*  
            TR-1 (Supports FR-1): Implement a password reset API endpoint (`/api/auth/reset-password`) that sends a time-limited tokenized link to the user’s registered email.  

         3. **Non-Functional Requirements (Optional but recommended):**
            - Capture reliability, usability, maintainability, scalability requirements if relevant.  

         ### Output Format
         Return the output in **well-structured Markdown** with the following sections:  

         - **Functional Requirements (FRs)**  
            - FR ID, Linked User Story, Description  

         - **Technical Requirements (TRs)**  
            - TR ID, Linked FR(s), Technical Details  

         - **Non-Functional Requirements (NFRs)** (if applicable)  

         ### Example Output

         **Functional Requirements**  
         - FR-1 (Linked to US-1): The system shall allow a registered user to reset their password securely.  
         - FR-2 (Linked to US-2): The system shall notify the user upon successful password reset.  

         **Technical Requirements**  
         - TR-1 (Supports FR-1): Provide an API endpoint `/api/auth/reset-password` that sends a one-time secure token via email.  
         - TR-2 (Supports FR-2): Use event-driven architecture with a message queue to trigger email notifications.  

         **Non-Functional Requirements**  
         - NFR-1: Password reset token must expire after 24 hours.  
         - NFR-2: The system shall support 10,000 concurrent password reset requests without downtime.  

         ---
         
         User Story:
         \n\n {user_story}
         Now, generate the **design documentation** for the given user stories.
        """,
        input_variables=["user_story"]
    )
    
    design_docs_chain = prompt_template | model | StrOutputParser()
    
    response = design_docs_chain.invoke({"user_story": state["auto_generated_user_stories_markdown"]})
    
    return {"design_docs": response}
    
    

In [15]:
def revise_user_stories():
    pass

In [16]:
def design_review():
    pass

In [17]:
def generate_code(state: SDLCState):
    prompt_template = PromptTemplate(
        template=""" 
            You are a highly skilled Software Engineer.  
            Your task is to generate production-ready source code based on the provided functional and technical requirements.  

            ### Input
            You will receive:
            - Functional Requirements (FRs) linked to user stories.  
            - Technical Requirements (TRs) that describe implementation details such as APIs, database design, security, and integrations.  

            ### Instructions
            1. Write clean, modular, and maintainable code that directly implements the given FRs and TRs.  
            2. Follow best practices for the specified programming language (naming conventions, error handling, code organization).  
            3. Respect the architectural style mentioned in TRs (e.g., microservices, MVC, layered architecture).  
            4. Apply security, validation, and reliability measures described in the requirements.  
            5. Use comments only where necessary to explain complex logic.  
            6. Return only the **source code**. Do not include explanations, test cases, or extra commentary.  

            ### Output Format
            - Provide the source code in properly formatted code blocks.  
            - If multiple files are required (e.g., API routes, models, configuration), clearly separate them with filenames as headers.  

            ---
            Technical and Functional design: \n\n
            {design_doc}
            Now, generate the complete source code according to the given functional and technical requirements.
        """,
        input_variables=["design_doc"],
    )

    generate_code_chain = prompt_template | model
    response = generate_code_chain.invoke({"design_doc": state["design_docs"]})
    
    return {"code": response.content}

In [18]:
def code_review(state: SDLCState):
    prompt_template = PromptTemplate(
        template="""
            You are acting as a Senior Software Engineer performing a code review.  
            Your task is to carefully evaluate the provided source code against the design requirements and best practices.  

            ### Input
            You will receive:
            - Source code generated from the approved design documents.  
            - Functional and Technical Requirements (FRs & TRs) that the code should implement.  

            ### Review Instructions
            When reviewing the code, check for the following aspects:

            1. Correctness  
            - Does the code correctly implement the functional and technical requirements?  
            - Are all functional flows handled as expected?  

            2. Code Quality  
            - Is the code clean, readable, and maintainable?  
            - Are naming conventions and coding standards followed?  
            - Is the logic modular and reusable (avoiding duplication)?  

            3. Error Handling & Reliability  
            - Are errors and exceptions handled gracefully?  
            - Is input validation included where necessary?  

            4. Performance & Maintainability  
            - Is the code efficient and reasonably optimized?  
            - Could the code be refactored for better maintainability?  

            ### Output Format
            Respond in one of the following ways **only**:

            - If the code is fully correct and ready → "approved"
            - If the code requires changes →  feedback 
                - [list each issue clearly and concisely in bullet points]


            ### Example Outputs

            **Approved case:**  
            "approved"


            **Feedback case:**  
            feedback:
                - Function process_data() does not handle empty input lists.
                - Variable names like tmp and val should be more descriptive.
                - Missing try/except for database connection.
                
        Code:
        \n\n {code}
        """,
        input_variables=["code"]
    )
    
    structured_llm = get_evaluator_schema(model)
    
    code_review_chain = prompt_template | structured_llm
    
    response = code_review_chain.invoke({"code": state["code"]})
    
    return {"code_review_response": response}

In [19]:
def fix_code_after_code_review(state: StateGraph) -> StateGraph:
    feedback = "\n\n".join(state["code_review_response"].feedback)

    prompt = config.get_prompt(
        "fix_code_after_code_review",
        original_code=state["code"],
        feedback=feedback,
    )

    fix_code_after_code_review_chain = model | StrOutputParser()
    response = fix_code_after_code_review_chain.invoke(prompt)

    return {"code": response}

In [20]:
def design_review_response(state: SDLCState) -> str:
    
    state_response = state["code_review_response"].status
    
    if state_response == "approved":
        return "approved"
    else:
        return "feedback"
    

In [21]:
def security_review(state: StateGraph) -> StateGraph:
    print("Aagya bhai mai security review me")
    return {"security_review_response": "Hello bhai"}

In [22]:
# THIS GRAPH IS USED FOR TESTING

graph = StateGraph(SDLCState)

graph.add_node("auto_generated_user_stories", auto_generated_user_stories)
graph.add_node("create_design_docs", create_design_docs)
graph.add_node("generate_code", generate_code)
graph.add_node("code_review", code_review)
graph.add_node("fix_code_after_code_review", fix_code_after_code_review)
graph.add_node("security_review", security_review)

graph.add_edge(START, "auto_generated_user_stories")
graph.add_edge("auto_generated_user_stories", "create_design_docs")
graph.add_edge("create_design_docs", "generate_code")
graph.add_edge("generate_code", "code_review")
graph.add_conditional_edges(
    "code_review",
    design_review_response,
    {"approved": "security_review", "feedback": "fix_code_after_code_review"},
)
graph.add_edge("security_review", END)
graph.add_edge("fix_code_after_code_review", END)

workflow = graph.compile()

In [23]:
response = workflow.invoke(
    {
        "user_input_requirements": """
        I want to build a task management web application. 
        Features:
        - Users can create, update, and delete tasks.
        - Tasks should have priority and due dates.
        - A dashboard to track completed vs pending tasks.
        - Simple login with email and password.
        Tech Stack Preference: Python (FastAPI) for backend, React for frontend.
        Deployment: Host on AWS.
        """
    }
)

BadRequestError: Error code: 400 - {'error': {'message': "tool call validation failed: parameters for tool EvaluatorSchema did not match schema: errors: [missing properties: 'status']", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<function=EvaluatorSchema> {"name": "EvaluatorSchema", "description": "Evaluates the provided source code against the design requirements and best practices.", "parameters": {"properties": {"feedback": {"anyOf": [{"items": {"type": "string"}, "type": "array"}, {"type": "null"}], "default": "null"}, "status": {"enum": ["approved", "feedback"], "type": "string"}},"required":["status"],"type":"object"}}</function>\n\nfeedback:\n- In `database.py`, the `SQLALCHEMY_DATABASE_URI` is set to an environment variable `DATABASE_URL`, but it is not checked if it exists before trying to connect to the database.\n- In `database.py`, the `engine` is created outside of a try/except block, but if the database connection fails, it will rais

In [ ]:
response

{'user_input_requirements': '\n        I want to build a task management web application. \n        Features:\n        - Users can create, update, and delete tasks.\n        - Tasks should have priority and due dates.\n        - A dashboard to track completed vs pending tasks.\n        - Simple login with email and password.\n        Tech Stack Preference: Python (FastAPI) for backend, React for frontend.\n        Deployment: Host on AWS.\n        ',
 'auto_generated_user_stories_markdown': [HumanMessage(content='### User Stories\n\n#### Task Creation and Management\n\nUS-1: As a registered user, I want to create a new task with priority and due date so that I can plan and organize my work efficiently.\n\n**Acceptance Criteria:**\n\n- Given that the user is logged in, when they click "Create Task", then they should see a form to input task details.\n- The user should be able to select a priority level (High, Medium, Low) and enter a due date.\n- The task should be saved in the database

### Remaining Nodes to code

In [ ]:
def write_test_case():
    pass

In [ ]:
def fix_code_after_security():
    pass

In [ ]:
def test_case_review():
    pass

In [ ]:
def qa_testing():
    pass

In [ ]:
def fix_test_cases_after_review():
    pass

In [ ]:
def deployment():
    pass

In [ ]:
def fix_code_after_qa_feedback():
    pass

### Conditional Branches

In [ ]:
def product_owner_response():
    response = "approved"
    if response == "approved":
        return response

In [ ]:
def code_review_response():
    response = "approved"
    if response == "approved":
        return response

In [ ]:
def security_review_reponse():
    response = "approved"
    if response == "approved":
        return response

In [ ]:
def test_case_review_response():
    response = "approved"
    if response == "approved":
        return response

In [ ]:
def qa_testing_response():
    response = "passed"
    if response == "passed":
        return response

### Create Graph, Add nodes and edges

In [ ]:
graph = StateGraph(SDLCState)

# Add Nodes
graph.add_node("user_input_requirements", user_input_requirements)
graph.add_node("auto_generated_user_stories", auto_generated_user_stories)
graph.add_node("product_owner_review", product_owner_review)
graph.add_node("create_design_docs", create_design_docs)
graph.add_node("revise_user_stories", revise_user_stories)
graph.add_node("design_review", design_review)
graph.add_node("generate_code", generate_code)
graph.add_node("code_review", code_review)
graph.add_node("fix_code_after_code_review", fix_code_after_code_review)
graph.add_node("security_review", security_review)
graph.add_node("write_test_case", write_test_case)
graph.add_node("fix_code_after_security", fix_code_after_security)
graph.add_node("test_case_review", test_case_review)
graph.add_node("qa_testing", qa_testing)
graph.add_node("fix_test_cases_after_review", fix_test_cases_after_review)
graph.add_node("deployment", deployment)
graph.add_node("fix_code_after_qa_feedback", fix_code_after_qa_feedback)


# Define Edges
graph.add_edge(START, "user_input_requirements")
graph.add_edge("user_input_requirements", "auto_generated_user_stories")
graph.add_edge("auto_generated_user_stories", "product_owner_review")
graph.add_conditional_edges(
    "product_owner_review",
    product_owner_response,
    {"approved": "create_design_docs", "feedback": "revise_user_stories"},
)
graph.add_edge(
    "revise_user_stories", "auto_generated_user_stories"
)  # When product_owner_response() is Feedback
graph.add_edge("create_design_docs", "design_review")

graph.add_conditional_edges(
    "design_review",
    design_review_response,
    {"approved": "generate_code", "feedback": "create_design_docs"},
)

graph.add_edge("generate_code", "code_review")
graph.add_conditional_edges(
    "code_review",
    code_review_response,
    {"approved": "security_review", "feedback": "fix_code_after_code_review"},
)
graph.add_edge("fix_code_after_code_review", "generate_code")

graph.add_conditional_edges(
    "security_review",
    security_review_reponse,
    {"approved": "write_test_case", "feedback": "fix_code_after_security"},
)
graph.add_edge("fix_code_after_security", "generate_code")

graph.add_edge("write_test_case", "test_case_review")
graph.add_conditional_edges(
    "test_case_review",
    test_case_review_response,
    {"approved": "qa_testing", "feedback": "fix_test_cases_after_review"},
)
graph.add_edge("fix_test_cases_after_review", "write_test_case")

graph.add_conditional_edges(
    "qa_testing",
    qa_testing_response,
    {"passed": "deployment", "failed": "fix_code_after_qa_feedback"},
)
graph.add_edge("fix_code_after_qa_feedback", "generate_code")

graph.add_edge("deployment", END)

In [ ]:
# Compile the graph
workflow = graph.compile()

### Misc

In [ ]:
"""
# Conditional Branches

1. product_owner_response()
2. design_review_response()
3. code_review_response()
4. security_review_reponse()
5. test_case_review_response()
6. qa_testing_response()
"""



"""
Nodes:

user_input_requirements()
auto_generated_user_stories()
product_owner_review()
create_design_docs() # Consist both functional & technical
revise_user_stories()
design_review()
generate_code()
code_review()
fix_code_after_code_review()
security_review()
write_test_case()
fix_code_after_security()
test_case_review()
qa_testing()
fix_test_cases_after_review()
deployment()
fix_code_after_qa_feedback()
"""

'\nNodes:\n\nuser_input_requirements()\nauto_generated_user_stories()\nproduct_owner_review()\ncreate_design_docs() # Consist both functional & technical\nrevise_user_stories()\ndesign_review()\ngenerate_code()\ncode_review()\nfix_code_after_code_review()\nsecurity_review()\nwrite_test_case()\nfix_code_after_security()\ntest_case_review()\nqa_testing()\nfix_test_cases_after_review()\ndeployment()\nfix_code_after_qa_feedback()\n'